In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,013 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1,998 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRele

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-03 15:31:21--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.78MB/s    in 0.2s    

2023-03-03 15:31:22 (5.78 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [17]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
import urllib.request


url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Outdoors_v1_00.tsv.gz'), sep="\t", header=True)

df.show()



+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [20]:
# Get the number of rows in the DataFrame.
num_rows = df.count()
print(f"Number of rows: {num_rows}")

Number of rows: 2302401


# Transform the Data

## Create the "review_id_table".

In [27]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.

# CREATE TABLE review_id_table (
#   review_id TEXT PRIMARY KEY NOT NULL,
#   customer_id INTEGER,
#   product_id TEXT,
#   product_parent INTEGER,
#   review_date DATE -- this should be in the formate yyyy-mm-dd
# );


review_id_table = df.selectExpr("cast(review_id as string) review_id",
    "cast(customer_id as int) customer_id",
    "cast(product_id as string) product_id",
    "cast(product_parent as int) product_parent",
    "cast(review_date as date) review_date")

for col in review_id_table.dtypes:
    print(col[0]+" , "+col[1])

review_id_table.show(5)


review_id , string
customer_id , int
product_id , string
product_parent , int
review_date , date
+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R35T75OLUGHL5C|   18446823|B000NV6H94|     110804376| 2015-08-31|
|R2BV735O46BN33|   13724367|B000IN0W3Y|     624096774| 2015-08-31|
|R2NBEUGPQQGXP1|   51001958|B008RBJXFM|     278970944| 2015-08-31|
|R17LLAOJ8ITK0S|   32866903|B00FK8WUQY|     312877650| 2015-08-31|
|R39PEQBT5ISEF4|   30907790|B00EZA3VW0|     305567912| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



## Create the "products" Table

In [30]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 

# CREATE TABLE products (
#   product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
#   product_title TEXT
# );

products_df = df.selectExpr("cast(product_id as string) product_id",
    "cast(product_title as string) product_title")

for col in products_df.dtypes:
    print(col[0]+" , "+col[1])

products_df.show(5)

product_id , string
+----------+
|product_id|
+----------+
|B000NV6H94|
|B000IN0W3Y|
|B008RBJXFM|
|B00FK8WUQY|
|B00EZA3VW0|
+----------+
only showing top 5 rows



## Create the "customers" Table

In [37]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
from pyspark.sql.functions import sum,avg,max,count

# CREATE TABLE customers (
#   customer_id INT PRIMARY KEY NOT NULL UNIQUE,
#   customer_count INT
# );


customers = df.groupBy("customer_id") \
    .agg(count("*").alias("customer_count")) \
    .show(truncate=False)




+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|43490052   |5             |
|38350414   |1             |
|10247520   |2             |
|51290010   |1             |
|35638967   |1             |
|3962209    |1             |
|11757653   |2             |
|29135708   |1             |
|30636778   |115           |
|46118365   |1             |
|43801871   |6             |
|14069091   |1             |
|5527036    |1             |
|50583551   |3             |
|13684282   |1             |
|32729496   |1             |
|45676353   |6             |
|25507311   |2             |
|22050232   |1             |
|30021189   |2             |
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [38]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
# -- vine table
# CREATE TABLE vine_table (
#   review_id TEXT PRIMARY KEY,
#   star_rating INTEGER,
#   helpful_votes INTEGER,
#   total_votes INTEGER,
#   vine TEXT
# );

vine = df.selectExpr("cast(review_id as string) review_id",
    "cast(star_rating as int) star_rating",
    "cast(helpful_votes as int) helpful_votes",
    "cast(total_votes as int) total_votes",
    "cast(vine as string) vine")

for col in vine.dtypes:
    print(col[0]+" , "+col[1])

vine.show(5)

review_id , string
star_rating , int
helpful_votes , int
total_votes , int
vine , string
+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R35T75OLUGHL5C|          4|            0|          0|   N|
|R2BV735O46BN33|          5|            0|          0|   N|
|R2NBEUGPQQGXP1|          4|            0|          0|   N|
|R17LLAOJ8ITK0S|          3|            1|          1|   N|
|R39PEQBT5ISEF4|          1|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



# Load

In [39]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products_df', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine.write.jdbc(url=jdbc_url, table='vine', mode=mode, properties=config)